In [1]:
import pandas as pd
import numpy as np
from clustering import get_cluster_data, generate_validation_plots
from clustering import clustering_experiment, generate_cluster_plots
from clustering import generate_component_plots
from helpers import get_abspath, save_array
from sklearn.decomposition import PCA
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn

winepath = get_abspath('winequality.csv', 'data/experiments')
seismicpath = get_abspath('seismic_bumps.csv', 'data/experiments')
wine = np.loadtxt(winepath, delimiter=',')
seismic = np.loadtxt(seismicpath, delimiter=',')

# set explained variance threshold
evp = 0.85

# split data into X and y
wX = wine[:, :-1]
wY = wine[:, -1]
sX = seismic[:, :-1]
sY = seismic[:, -1]